In [ ]:
!pip install torch_geometric --quiet

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import os
import json
from torch_geometric.utils import dense_to_sparse
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import OPTICS, DBSCAN
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader

# Custom imports
import sys
sys.path.insert(1, "/kaggle/input/second-dataset/dependecies")

import GCN
from Graph import Graph
from data_generator import generate_arch_dicts

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TEST = False

In [2]:
model_diversity_path = "weights/model_diversity_weights.pth"
model_accuracy_path = "weights/model_accuracy_weights.pth"

In [3]:
input_dim = 8
output_dim = 16
dropout=0
pooling = "mean"

model_diverisity = GCN.GCN(input_dim, output_dim, dropout, pooling).to(device)
state_dict = torch.load(model_diversity_path, map_location=device, weights_only=True)
model_diverisity.load_state_dict(state_dict)
model_diverisity.eval()

GCN(
  (gc1): GCNConv(8, 64)
  (gc2): GCNConv(64, 256)
  (gc3): GCNConv(256, 512)
  (gc4): GCNConv(512, 64)
  (residual_proj): Linear(in_features=8, out_features=64, bias=True)
  (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0, inplace=False)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
)

In [4]:
input_dim = 8
output_dim = 1
dropout = 0
pooling = "mean"

model_accuracy = GCN.GCN(input_dim, output_dim, dropout, pooling).to(device)
state_dict = torch.load(model_accuracy_path, map_location=device, weights_only=True)
model_accuracy.load_state_dict(state_dict)
model_accuracy.eval()

GCN(
  (gc1): GCNConv(8, 64)
  (gc2): GCNConv(64, 256)
  (gc3): GCNConv(256, 512)
  (gc4): GCNConv(512, 64)
  (residual_proj): Linear(in_features=8, out_features=64, bias=True)
  (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0, inplace=False)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

Зададим гиперпараметры:

1. K -- количество моделей в ансамбле
2. ALPHA -- трейд-off между точностью и разнообразием, где разнообразие определяется как расстояние эмбеддингов до ансамбля.
3. N -- количество моделей в одной эпохе

Мы хотим, чтобы в ансамбле были точные модели, поэтому введем $\gamma$ -- минимальную точность модели.

In [5]:
K = 6
ALPHA = 0.9
N = 50000
GAMMA = 0.9
BATCH_SIZE = 128

In [6]:
best_models = []
potential_archs = []

while len(best_models) < K:
    arch_dicts = generate_arch_dicts(N, use_tqdm=True)
    arch_dicts = np.array(arch_dicts)
    graphs = [Graph(arch_dict, index=i) for i, arch_dict in enumerate(arch_dicts)]
    dataset = GCN.CustomDataset(graphs, use_tqdm=True)
    loader_accuracy = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=4,
        collate_fn=GCN.collate_graphs,
    )

    embeddings, indices = GCN.extract_embeddings(
        model_accuracy, loader_accuracy, device, use_tqdm=True
    )
    appropriate_indices = indices[embeddings > GAMMA]
    potential_archs.extend(arch_dicts[appropriate_indices])

    accuracy_quantile = np.quantile(embeddings, 0.975)

    print("0.975 accuracy quantile:", accuracy_quantile)
    print("Max accuracy:", np.max(embeddings))

    if len(appropriate_indices) > 0:
        print(potential_archs[-1])
        break

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 8/50000 [00:05<9:17:04,  1.50it/s]/home/alexander/.local/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
100%|██████████| 391/391 [00:48<00:00,  8.08it/s]

0.975 accuracy quantile: 0.866715955734253
Max accuracy: 0.88369054


  0%|          | 0/50000 [00:00<?, ?it/s]

100%|██████████| 391/391 [01:05<00:00,  6.01it/s]


0.975 accuracy quantile: 0.8667777433991433
Max accuracy: 0.882653


  0%|          | 0/50000 [00:00<?, ?it/s]

KeyboardInterrupt: 